In [259]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [260]:
date = "D:/kaggle_date/美国大学生篮球比赛/march-machine-learning-mania-2024/"

#### 整理出每只队伍的胜率

In [261]:
#### 
df_seeds = pd.concat([
    pd.read_csv(date + "MNCAATourneySeeds.csv"),
    pd.read_csv(date + "WNCAATourneySeeds.csv"),
], ignore_index=True)
df_seeds_2023 = df_seeds[df_seeds['Season'] == 2023]
# df_seeds_2023

In [262]:
df_season_results = pd.concat([
    pd.read_csv(date + "MRegularSeasonCompactResults.csv"),
    pd.read_csv(date + "WRegularSeasonCompactResults.csv"),
], ignore_index=True)
df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

df_season_results["score"] = df_season_results["WScore"] - df_season_results["LScore"]

In [263]:
df_season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,score
0,1985,20,1228,81,1328,64,17
1,1985,25,1106,77,1354,70,7
2,1985,25,1112,63,1223,56,7
3,1985,25,1165,70,1432,54,16
4,1985,25,1192,86,1447,74,12
...,...,...,...,...,...,...,...
316095,2024,100,3409,74,3427,70,4
316096,2024,100,3424,67,3161,64,3
316097,2024,100,3444,65,3296,47,18
316098,2024,100,3457,54,3347,51,3


In [264]:
df_win = df_season_results.groupby(["Season","WTeamID"]).count().reset_index()[["Season","WTeamID","DayNum"]].rename(columns={"WTeamID":"TeamID","DayNum":"numwin"})
df_loss = df_season_results.groupby(["Season","LTeamID"]).count().reset_index()[["Season","LTeamID","DayNum"]].rename(columns={"LTeamID":"TeamID","DayNum":"numloss"})

In [265]:
###计算平均获胜得分
gap_win = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()
gap_win = gap_win[['Season', 'WTeamID', 'score']].rename(columns={"score": "gapWin", "WTeamID": "TeamID"})
###计算平均失败得分
gap_loss = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()
gap_loss = gap_loss[['Season', 'LTeamID', 'score']].rename(columns={"score": "gaploss", "LTeamID": "TeamID"})

In [266]:
### 获胜队伍和失败队伍
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})
df_features_season = pd.concat([df_features_season_w, df_features_season_l], axis=0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)##合并并删除重复行和重复索引

In [267]:
df_features_season = df_features_season.merge(df_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(df_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_loss, on=['Season', 'TeamID'], how='left')
df_features_season.fillna(0, inplace=True)  

In [268]:
df_features_season["p_win"] = df_features_season["numwin"]/(df_features_season["numloss"]+df_features_season["numwin"])

In [269]:
df_features_season["avggap"] = (df_features_season["gapWin"]*df_features_season["numwin"]- df_features_season["gaploss"]*df_features_season["numloss"])/(df_features_season["numloss"]+df_features_season["numwin"])

In [270]:
df_features_season.drop(["numwin","numloss","gapWin","gaploss"],axis=1,inplace=True)

In [271]:
# 比赛胜率
df_features_season###每一年的比赛胜率和得分

,Season,TeamID,p_win,avggap
0,1985,1102,0.208333,-5.791667
1,1985,1103,0.391304,-3.043478
2,1985,1104,0.700000,7.800000
3,1985,1106,0.416667,-3.791667
4,1985,1108,0.760000,7.960000
...,...,...,...,...
22145,2024,3474,0.142857,-13.952381
22146,2024,3475,0.700000,3.150000
22147,2024,3476,0.130435,-17.695652
22148,2024,3477,0.450000,-7.600000


#### 球员实力

In [272]:
Result_m = pd.read_csv(date + "MNCAATourneyDetailedResults.csv")

In [273]:
Result_m.iloc[:,0]

0       2003
1       2003
2       2003
3       2003
4       2003
        ... 
1310    2023
1311    2023
1312    2023
1313    2023
1314    2023
Name: Season, Length: 1315, dtype: int64

In [274]:
win_team =  pd.concat([Result_m.iloc[:,0],Result_m.iloc[:,2:4],Result_m.iloc[:,21:]],axis=1)
loss_team =  pd.concat([Result_m.iloc[:,0],Result_m.iloc[:,4:6],Result_m.iloc[:,8:21]],axis=1)

In [275]:
win_num = win_team.groupby(["Season","WTeamID"]).count().reset_index()["WScore"]
loss_num = loss_team.groupby(["Season","LTeamID"]).count().reset_index()["LScore"]

In [276]:
win_result = win_team.groupby(["Season","WTeamID"]).mean().reset_index().rename(columns={"WTeamID":"TeamID"})
loss_result = loss_team.groupby(["Season","LTeamID"]).mean().reset_index().rename(columns={"LTeamID":"TeamID"})
result = win_result.merge(loss_result, on=['Season', 'TeamID'], how='left')

In [277]:
# result["p_win"] = win_team/win_team

In [279]:
# loss_team.groupby(["Season","WTeamID"]).mean().reset_index()

In [280]:
Result_w = pd.read_csv(date + "WNCAATourneyDetailedResults.csv")

In [281]:
win_team =  pd.concat([Result_w.iloc[:,0],Result_w.iloc[:,2:4],Result_w.iloc[:,21:]],axis=1)
loss_team =  pd.concat([Result_w.iloc[:,0],Result_w.iloc[:,4:6],Result_w.iloc[:,8:21]],axis=1)

In [282]:
win_result = win_team.groupby(["Season","WTeamID"]).mean().reset_index().rename(columns={"WTeamID":"TeamID"})
loss_result = loss_team.groupby(["Season","LTeamID"]).mean().reset_index().rename(columns={"LTeamID":"TeamID"})
result_w = win_result.merge(loss_result, on=['Season', 'TeamID'], how='left')

In [283]:
result_all = pd.concat([result,result_w],ignore_index=True)

In [284]:
result_all.drop(columns=["WScore","LScore"],axis=1,inplace=True)

In [285]:
team_a = result_all.merge(df_features_season, on=['Season', 'TeamID'], how='left')

In [286]:
team_a = team_a.fillna(0)

In [287]:
team_a.iloc[:,2:15]

,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,26.333333,66.666667,8.333333,23.0,11.333333,15.333333,11.0,26.666667,15.666667,15.666667,7.00,4.0,18.666667
1,25.000000,69.000000,7.000000,28.0,14.000000,21.000000,20.0,22.000000,11.000000,12.000000,2.00,5.0,18.000000
2,20.000000,54.000000,7.000000,23.5,15.500000,18.500000,13.5,22.000000,8.000000,15.000000,7.50,5.5,21.000000
3,22.000000,49.000000,8.500000,23.0,6.000000,9.000000,8.0,20.500000,11.500000,12.000000,7.00,3.0,17.000000
4,27.000000,60.000000,7.000000,17.0,12.000000,17.000000,14.0,17.000000,20.000000,21.000000,6.00,6.0,21.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,24.000000,69.000000,6.000000,31.0,19.000000,22.000000,11.0,29.000000,11.000000,8.000000,5.00,0.0,18.000000
1103,20.500000,54.000000,5.500000,15.5,12.500000,19.500000,4.5,20.500000,13.500000,14.000000,3.00,3.0,18.000000
1104,24.000000,70.500000,6.000000,24.0,12.500000,16.000000,13.0,17.000000,10.500000,13.500000,9.00,4.5,24.500000
1105,19.500000,56.000000,5.500000,20.0,13.500000,15.500000,7.0,19.500000,10.000000,13.000000,4.00,2.0,17.000000


In [288]:
team_a.iloc[:,2:15] = (-1*(np.array(team_a.iloc[:,15:28])+np.array(team_a.iloc[:,2:15])))/np.array(team_a.iloc[:,15:28].isnull()-2)

In [289]:
team_a.drop(columns=team_a.columns[15:28],inplace=True)

In [290]:
new_c_n = {}
for i in range(2,15):
    new_c_n[team_a.columns[i]] = team_a.columns[i][1:]

In [291]:
team_a = team_a.rename(columns=new_c_n)

In [292]:
# mseason = pd.read_csv(date+"MSeasons.csv")
# wseason = pd.read_csv(date+"WSeasons.csv")

In [293]:
# season = pd.concat([wseason,mseason],ignore_index=True)
# season.drop(columns=["DayZero"],inplace=True)

In [294]:
# team_a = team_a.merge(season,on=["Season"],how="left")

In [295]:
# df_features_season =  df_features_season.merge(season,on=["Season"],how="left")

In [301]:
m_seed =pd.read_csv(date + "MRegularSeasonCompactResults.csv")
w_seed  =pd.read_csv(date + "WRegularSeasonCompactResults.csv")

In [299]:
# m_seed["seed"] = 1
# w_seed["seed"] = 0
# m_seed.drop(columns=["Seed","Season"],inplace=True)
# w_seed.drop(columns=["Seed","Season"],inplace=True)

In [302]:
id = np.unique(list(m_seed["WTeamID"]) + list(m_seed["LTeamID"]))
m_seed = pd.DataFrame(np.c_[id,np.ones(len(id))],columns=["TeamID","seed"])
wid = np.unique(list(w_seed["WTeamID"]) + list(w_seed["LTeamID"]))
w_seed = pd.DataFrame(np.c_[wid,np.ones(len(wid))],columns=["TeamID","seed"])

In [303]:
team_a=team_a.merge(pd.concat([m_seed,w_seed],ignore_index=True),on=["TeamID"],how="left")

In [304]:
df_features_season =df_features_season.merge(pd.concat([m_seed,w_seed],ignore_index=True),on=["TeamID"],how="left")

In [305]:
# team_a.drop(columns=["RegionX","RegionW","RegionY","RegionZ"],inplace=True)
# df_features_season.drop(columns=["RegionX","RegionW","RegionY","RegionZ"],inplace=True)

In [336]:
# city_m = pd.read_csv(date+"MGameCities.csv")
# city_w = pd.read_csv(date+"WGameCities.csv")

In [357]:
team_a[team_a["seed"] == 1].iloc[:,:15]

,Season,TeamID,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2003,1112,27.666667,66.833333,8.166667,24.50,11.666667,16.666667,13.00,24.333333,15.833333,15.333333,10.000,4.50,19.333333
1,2003,1113,32.500000,64.000000,6.000000,18.50,18.500000,27.500000,16.00,25.000000,16.000000,12.500000,4.000,5.50,19.000000
2,2003,1120,24.500000,60.000000,6.000000,18.75,15.750000,19.750000,14.25,21.500000,11.000000,12.500000,8.250,6.25,19.000000
3,2003,1139,24.500000,52.000000,6.250000,16.50,6.500000,10.500000,13.50,20.250000,13.250000,12.500000,6.500,2.00,17.500000
4,2003,1141,29.500000,56.000000,8.500000,16.00,12.000000,16.500000,9.00,20.000000,17.000000,16.000000,10.500,7.50,19.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,2023,3405,29.500000,72.500000,8.500000,26.50,16.000000,20.000000,13.00,33.000000,15.500000,10.000000,7.000,1.50,17.000000
1103,2023,3417,22.250000,58.500000,4.750000,15.75,9.750000,15.750000,9.25,22.250000,16.250000,11.500000,4.000,4.50,14.500000
1104,2023,3428,24.000000,64.250000,4.500000,18.50,13.750000,19.500000,11.50,19.000000,11.750000,13.250000,8.500,4.75,23.250000
1105,2023,3437,22.750000,56.500000,4.250000,15.00,14.250000,16.250000,9.50,24.250000,11.000000,15.000000,6.000,1.00,17.500000


In [359]:
m_date = df_features_season[df_features_season["seed"]==1].merge(team_a[team_a["seed"] == 1].iloc[:,:15],on=["Season","TeamID"],how="left")

In [388]:
m_date = m_date.fillna(0)

In [399]:
mm_date = df_features_season[df_features_season["seed"]==1].iloc[:,:4]

In [475]:
end_date = np.zeros([len(np.unique(m_date["TeamID"])),40,4])

In [476]:
id_lst = np.unique(m_date["TeamID"])
for i in range(len(id_lst)):
    mmm_date = mm_date[mm_date["TeamID"] == id_lst[i]]
    for j in range(0,40):
        dd = mmm_date[mmm_date["Season"] == j+1985]
        if not dd.empty:
           end_date[i,j] = np.array(dd)    
        else:
           end_date[i,j] = [j+1985,id_lst[i],0,0]  

In [505]:
print(end_date[:-200,:,1:][:,1:].shape)
print(end_date[:-200,:,1:][:,:-1].shape)

(545, 39, 3)
(545, 39, 3)


In [578]:
lstm = nn.LSTM(3, 3, 3, batch_first=True)

In [579]:
qq,ww = lstm(x)

In [765]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.fc(out[:, -1, :])
        # out = self.fc(out)
        
        return out

# # 定义模型输入维度和超参数
# input_size = 3
# hidden_size = 64
# num_layers = 1
# output_size = 3

# 初始化模型
# model = LSTM(input_size, hidden_size, num_layers, output_size)

# # 打印模型结构
# print(model)


In [766]:
x = torch.tensor(end_date[:-100,:-1,2:],dtype=torch.float32)
y = torch.tensor(end_date[:-100,-1,2:],dtype=torch.float32)

In [835]:
input_size = 2
hidden_size = 64
num_layers = 2
output_size = 2
num_epochs = 150
model = LSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(num_epochs):
    # 定义模型输入维度和超参数
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/150], Loss: 32.795841217041016
Epoch [2/150], Loss: 31.3453369140625
Epoch [3/150], Loss: 29.256120681762695
Epoch [4/150], Loss: 27.149829864501953
Epoch [5/150], Loss: 26.302343368530273
Epoch [6/150], Loss: 25.92909049987793
Epoch [7/150], Loss: 25.460023880004883
Epoch [8/150], Loss: 24.835159301757812
Epoch [9/150], Loss: 24.634214401245117
Epoch [10/150], Loss: 23.51064682006836
Epoch [11/150], Loss: 22.628658294677734
Epoch [12/150], Loss: 21.700397491455078
Epoch [13/150], Loss: 21.00874900817871
Epoch [14/150], Loss: 20.524200439453125
Epoch [15/150], Loss: 20.029827117919922
Epoch [16/150], Loss: 19.458709716796875
Epoch [17/150], Loss: 18.951875686645508
Epoch [18/150], Loss: 18.59408950805664
Epoch [19/150], Loss: 18.375871658325195
Epoch [20/150], Loss: 18.326784133911133
Epoch [21/150], Loss: 18.250577926635742
Epoch [22/150], Loss: 18.06178092956543
Epoch [23/150], Loss: 17.82497215270996
Epoch [24/150], Loss: 17.691242218017578
Epoch [25/150], Loss: 17.56529617

In [836]:
num_epochs = 200
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(num_epochs):
    # 定义模型输入维度和超参数
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/200], Loss: 0.09687228500843048
Epoch [2/200], Loss: 0.4266519248485565
Epoch [3/200], Loss: 0.14734989404678345
Epoch [4/200], Loss: 0.24028414487838745
Epoch [5/200], Loss: 0.24423450231552124
Epoch [6/200], Loss: 0.19899876415729523
Epoch [7/200], Loss: 0.16809187829494476
Epoch [8/200], Loss: 0.15143106877803802
Epoch [9/200], Loss: 0.15027371048927307
Epoch [10/200], Loss: 0.15487754344940186
Epoch [11/200], Loss: 0.1536681354045868
Epoch [12/200], Loss: 0.14402823150157928
Epoch [13/200], Loss: 0.13080912828445435
Epoch [14/200], Loss: 0.1226370558142662
Epoch [15/200], Loss: 0.1189068928360939
Epoch [16/200], Loss: 0.1183086410164833
Epoch [17/200], Loss: 0.1192917674779892
Epoch [18/200], Loss: 0.11953405290842056
Epoch [19/200], Loss: 0.11694175004959106
Epoch [20/200], Loss: 0.11165381222963333
Epoch [21/200], Loss: 0.10613365471363068
Epoch [22/200], Loss: 0.10303519666194916
Epoch [23/200], Loss: 0.10293351858854294
Epoch [24/200], Loss: 0.10353139042854309
Epoch [

In [837]:
# # # 使用模型进行预测
model.eval()
X_test = torch.tensor(end_date[-100:,:-1,2:], dtype=torch.float32)
predictions = model(X_test)

In [838]:
((predictions - torch.tensor(end_date[-100:,-1,2:], dtype=torch.float32))**2).mean()

tensor(37.0198, grad_fn=<MeanBackward0>)

In [839]:
end_date[5,-1,2:]

array([ 0.38095238, -5.71428571])

In [834]:
predictions[5]

tensor([0.3872, 0.8090], grad_fn=<SelectBackward0>)

In [1]:
t1 = {1:222,2:333,3:444}

dict_values([222, 333, 444])